## TensorFlow Recurrent Neural Networks (Complete guide with examples and code)

Click the image below to read the post online.

<a target="_blank" href="https://www.machinelearningnuggets.com/tensorflow-lstm
"><img src="https://www.machinelearningnuggets.com/ezoimgfmt/digitalpress.fra1.cdn.digitaloceanspaces.com/mhujhsj/2022/07/logho-1.png?ezimgfmt=ng:webp/ngcb1" alt="Open in ML Nuggets"></a>

In [ ]:
# !pip install tensorflow

In [ ]:
# !pip install keras

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Bidirectional
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [ ]:
pip install wget

In [ ]:
import wget
wget.download("https://ml.machinelearningnuggets.com/loans.csv")

In [ ]:
###Import the data
loans = pd.read_csv("loans.csv")

In [ ]:
loans = loans[['created_at','amount']]

In [ ]:
loans['created_at'] = pd.DatetimeIndex(loans['created_at'])

In [ ]:
loans = loans.groupby(['created_at']).amount.sum().reset_index()

In [ ]:
loans.sort_values(by=['created_at'], inplace=True)

In [ ]:
loans = loans.set_index('created_at')

In [ ]:
loans

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_loans = scaler.fit_transform(loans)

In [ ]:
scaled_loans

In [ ]:
x_train = []
y_train = []
for i in range(60, 1955):
    x_train.append(scaled_loans[i-60:i, 0])
    y_train.append(scaled_loans[i, 0])

In [ ]:
y_train = np.array(y_train)
x_train = np.array(x_train)

In [ ]:
x_train = np.reshape(x_train, (np.shape(x_train)[0], np.shape(x_train)[1], 1 ))

In [ ]:
regressor  = Sequential()
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(np.shape(x_train)[1],1)))
regressor.add(Dropout(0.2))

regressor.add(Bidirectional(LSTM(units=50, return_sequences=True)))
regressor.add(Dropout(0.2))

regressor.add(Bidirectional(LSTM(units=50, return_sequences=True)))
regressor.add(Dropout(0.2))

regressor.add(Bidirectional(LSTM(units=50)))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))
regressor.add(Dropout(0.2))

regressor.add(Dense(units=1))

regressor.summary()

In [ ]:
lr_schedule = keras.callbacks.LearningRateScheduler(lambda epoch:1e-7 * 10**(epoch/20))

opt = tf.keras.optimizers.Adam(learning_rate=1e-7)

regressor.compile(optimizer=opt, loss='mse', metrics=['mae','mape'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min',patience=20)

mc = tf.keras.callbacks.ModelCheckpoint('best_model.h5', monitor='loss', mode='min', verbose=0, save_best_only=True)

hist = regressor.fit(x_train,y_train, epochs=150, batch_size=32, callbacks=[mc, lr_schedule, early_stopping])


In [ ]:
print(hist.history.keys())


In [ ]:
plt.plot(hist.history['loss'])

In [ ]:
y_preds = regressor.predict(x_train)

In [ ]:
y_preds

In [ ]:
def getError(actual, prediction):
  m = keras.metrics.MeanAbsolutePercentageError()
  n = keras.metrics.MeanAbsoluteError()
  m.update_state(actual, prediction)
  n.update_state(actual, prediction)
  err = m.result().numpy()
  err_1 = n.result().numpy()
  return ({'MAE':err_1, 'MAPE':err})

In [ ]:
train_errors = getError(y_train, y_preds)
print(train_errors)

In [ ]:
prediction = regressor.predict( np.array( [x_train[-1],]))

In [ ]:
print(prediction)

In [ ]:
x_copy = x_train
y_copy = y_train

In [ ]:
def predictSteps(x, steps):
  if(steps == 0):
    prediction = regressor.predict( np.array( [x,]))
    print(prediction)
    return prediction
  else:
    prediction = regressor.predict( np.array( [x,]))
    pred = x[1:]
    pred = np.append(pred,prediction)
    steps = steps-1
    print(prediction)
    predictSteps(pred, steps)

In [ ]:
predictSteps(x_copy[-1], 10)

## Where to go from here
Follow us on [LinkedIn](https://www.linkedin.com/company/mlnuggets), [Twitter](https://twitter.com/ml_nuggets), [GitHub](https://github.com/mlnuggets) and subscribe to our [blog](https://www.machinelearningnuggets.com/#/portal) so that you don't miss a new issue.